## Looking at correlations between the same metrics across different core logs:

In [2]:
import os
import time
import pandas as pd
import numpy as np

In [3]:
import eigen_analysis

cols = eigen_analysis.LINUX_COLS
time_unit = eigen_analysis.TIME_CONVERSION_khz
joules_unit = eigen_analysis.JOULE_CONVERSION

In [33]:
def prep_counters_df(df, start, end):
    df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
    df.loc[:, 'timestamp'] = df['timestamp'] - df['timestamp'].min()
    df.loc[:, 'timestamp'] = df['timestamp'] * time_unit
    df.loc[:, 'joules'] = df['joules'] * joules_unit

    # removing zero-filled log-entries
    # -> these represent interrupt occurrences at a frequency greater than per-1ms
    counters_df = df[['joules', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'timestamp']].copy()
    counters_df = counters_df[(counters_df['joules'] > 0) & (counters_df['instructions'] > 0) \
                                        & (counters_df['cycles'] > 0) & (counters_df['ref_cycles'] > 0) \
                                        & (counters_df['llc_miss'] > 0)]
    return counters_df

In [80]:
qps = '200k'
run = '0'
dvfs = '0xd00'
rapl = '135'
itr = '100'

logs_dir = qps + '_qps/linux_mcd_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
rdtsc_dir = qps + '_qps/linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
start, end = eigen_analysis.get_rdtsc(rdtsc_file)

df_inst = pd.DataFrame()
for c in range(0, 16):
    core = str(c)
    c_log_file = logs_dir + 'linux.mcd.dmesg.0_' + core + '_100_0xd00_135_200000'
    df = pd.read_csv(c_log_file, sep = ' ', names=cols, index_col='i')
    df_counters = prep_counters_df(df, start, end)
    df_inst_c = df_counters[['instructions', 'timestamp']].copy()
    if df_inst.shape[0] == 0:
        df_inst = df_inst_c
    else:
        df_inst = df_inst.merge(df_inst_c, on='timestamp', \
                         how='outer', sort=True, suffixes=('', '_0')).fillna(0)

In [81]:
df_inst

,instructions,timestamp,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0
0,0.000000e+00,0.000000,7.686818e+12,0.000000e+00,7.676870e+12,7.522764e+12,0.000000e+00,7.438144e+12,0.000000e+00,7.500489e+12,7.682446e+12,7.523976e+12,0.000000e+00,7.513715e+12,7.687445e+12,7.550595e+12,7.703373e+12
1,0.000000e+00,0.000267,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.663011e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000000e+00,0.000300,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.688966e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.000000e+00,0.000304,0.000000e+00,7.512387e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,7.548044e+12,0.000319,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307824,0.000000e+00,20.003593,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.688587e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
307825,0.000000e+00,20.003672,7.692975e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
307826,0.000000e+00,20.003673,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.530044e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
307827,0.000000e+00,20.003709,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.685489e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [82]:
df_inst.drop(['timestamp'], axis = 1).corr()

,instructions,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0,instructions_0
instructions,1.000000,-0.066678,-0.066714,-0.066680,-0.066690,-0.066636,-0.066717,-0.066660,-0.066717,-0.066673,-0.066686,-0.066669,-0.066678,-0.066682,-0.066690,-0.066673
instructions_0,-0.066678,1.000000,-0.066686,-0.066597,-0.066606,-0.066664,-0.066634,-0.066632,-0.066634,-0.066590,-0.066603,-0.066641,-0.066595,-0.066599,-0.066606,-0.066590
instructions_0,-0.066714,-0.066686,1.000000,-0.066688,-0.066697,-0.066699,-0.066725,-0.066667,-0.066725,-0.066680,-0.066693,-0.066677,-0.066686,-0.066690,-0.066697,-0.066680
instructions_0,-0.066680,-0.066597,-0.066688,1.000000,-0.066608,-0.066666,-0.066636,-0.066634,-0.066636,-0.066592,-0.066605,-0.066643,-0.066597,-0.066601,-0.066608,-0.066592
instructions_0,-0.066690,-0.066606,-0.066697,-0.066608,1.000000,-0.066675,-0.066645,-0.066643,-0.066645,-0.066601,-0.066614,-0.066653,-0.066606,-0.066610,-0.066618,-0.066601
instructions_0,-0.066636,-0.066664,-0.066699,-0.066666,-0.066675,1.000000,-0.066702,-0.066645,-0.066702,-0.066658,-0.066671,-0.066654,-0.066664,-0.066667,-0.066675,-0.066658
instructions_0,-0.066717,-0.066634,-0.066725,-0.066636,-0.066645,-0.066702,1.000000,-0.066671,-0.066673,-0.066629,-0.066642,-0.066680,-0.066634,-0.066638,-0.066645,-0.066629
instructions_0,-0.066660,-0.066632,-0.066667,-0.066634,-0.066643,-0.066645,-0.066671,1.000000,-0.066671,-0.066627,-0.066640,-0.066623,-0.066632,-0.066636,-0.066643,-0.066627
instructions_0,-0.066717,-0.066634,-0.066725,-0.066636,-0.066645,-0.066702,-0.066673,-0.066671,1.000000,-0.066629,-0.066642,-0.066680,-0.066634,-0.066638,-0.066645,-0.066629
instructions_0,-0.066673,-0.066590,-0.066680,-0.066592,-0.066601,-0.066658,-0.066629,-0.066627,-0.066629,1.000000,-0.066597,-0.066636,-0.066590,-0.066593,-0.066601,-0.066584


In [ ]:
from numpy.linalg import eig, eigvalsh

vals, vecs = 

### Test on 2 cores

In [83]:
c0_log_file = logs_dir + 'linux.mcd.dmesg.0_0_100_0xd00_135_200000'
c1_log_file = logs_dir + 'linux.mcd.dmesg.0_1_100_0xd00_135_200000'

df0 = pd.read_csv(c0_log_file, sep = ' ', names=cols, index_col='i')
df1 = pd.read_csv(c1_log_file, sep = ' ', names=cols, index_col='i')

df0_counters = prep_counters_df(df0, start, end)
df1_counters = prep_counters_df(df1, start, end)

df0_inst = df0_counters[['instructions', 'timestamp']].copy()
df1_inst = df1_counters[['instructions', 'timestamp']].copy()

In [84]:
df1_inst

,instructions,timestamp
i,,
133,7686818085571,0.000000
143,7686818632656,0.001035
153,7686819093449,0.002047
163,7686819510702,0.003086
173,7686819808421,0.004156
...,...,...
188485,7692972769333,19.999252
188493,7692972967446,20.000586
188503,7692973576699,20.001609


In [85]:
df_inst = pd.DataFrame()
df_inst = df0_inst.merge(df1_inst, on='timestamp', \
                         how='outer', sort=True, suffixes=('', '_0')).fillna(0)

In [86]:
df_inst

,instructions,timestamp,instructions_0
0,0.000000e+00,0.000000,7.686818e+12
1,7.548044e+12,0.000319,0.000000e+00
2,0.000000e+00,0.001035,7.686819e+12
3,7.548044e+12,0.001417,0.000000e+00
4,0.000000e+00,0.002047,7.686819e+12
...,...,...,...
38480,0.000000e+00,20.001609,7.692974e+12
38481,7.554094e+12,20.002006,0.000000e+00
38482,0.000000e+00,20.002634,7.692974e+12
38483,7.554095e+12,20.003100,0.000000e+00


In [87]:
df_inst.drop(['timestamp'], axis = 1).corr()

,instructions,instructions_0
instructions,1.0,-1.0
instructions_0,-1.0,1.0
